In [94]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [95]:
ratings = pd.read_csv("ratings_small.csv")
movies = pd.read_csv("movies_metadata.csv",dtype='unicode')
movies = movies[pd.to_numeric(movies["id"], errors='coerce').notnull()]
movies["id"] = movies["id"].astype(int)
ratings = pd.merge(ratings, movies, left_on='movieId', right_on='id').drop(columns=["timestamp", "budget","homepage","release_date","release_date","revenue","runtime","spoken_languages","status","poster_path","video","production_countries","production_companies","original_language","imdb_id","belongs_to_collection","tagline","original_title","genres","adult","id","overview","popularity","vote_average","vote_count"])
ratings = ratings.sort_values(by=['movieId','userId'])
ratings.head()

,userId,movieId,rating,title
22417,15,2,2.0,Ariel
22418,19,2,3.0,Ariel
22419,30,2,2.0,Ariel
22420,32,2,4.0,Ariel
22421,41,2,3.5,Ariel


In [96]:
user_ratings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
user_ratings

title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,...And God Created Woman,00 Schneider - Jagd auf Nihil Baxter,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 Angry Men,...,Zodiac,Zombie Flesh Eaters,Zombie Holocaust,Zozo,eXistenZ,xXx,¡Three Amigos!,À nos amours,Ödipussi,Şaban Oğlu Şaban
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
user_ratings = user_ratings.dropna(thresh=5,axis=1).fillna(0)
user_ratings

title,10 Items or Less,10 Things I Hate About You,15 Minutes,1984,2 Days in Paris,"20,000 Leagues Under the Sea",2001: A Space Odyssey,24 Hour Party People,25th Hour,28 Days Later,...,Young Adam,Young Frankenstein,Young and Innocent,Z,Zatoichi,Zazie dans le métro,Zodiac,eXistenZ,xXx,À nos amours
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [98]:
print("Movies left:")
for i in user_ratings:
    print(i,end=", ")

Movies left:
10 Items or Less, 10 Things I Hate About You, 15 Minutes, 1984, 2 Days in Paris, 20,000 Leagues Under the Sea, 2001: A Space Odyssey, 24 Hour Party People, 25th Hour, 28 Days Later, 28 Weeks Later, 3-Iron, 300, 4 Months, 3 Weeks and 2 Days, 40 Days and 40 Nights, 42nd Street, 48 Hrs., 5 Card Stud, 54, 7 Virgins, 8 Mile, 8 Women, 88 Minutes, 8½, 9 Songs, 99 francs, A Beautiful Mind, A Bridge Too Far, A Brief History of Time, A Clockwork Orange, A Close Shave, A Cure for Love, A Dandy in Aspic, A Friend of Mine, A Good Marriage, A Hard Day's Night, A Kiss Before Dying, A Night in Heaven, A Nightmare on Elm Street, A Perfect Murder, A River Runs Through It, A Shot in the Dark, A Star Is Born, A Tale of Two Cities, A Taxing Woman, A Time to Kill, A Very Long Engagement, A View to a Kill, A Woman, a Gun and a Noodle Shop, Abraham, Absolute Power, Aces Go Places V: The Terracotta Hit, Adam's Apples, Adaptation., Addicted to Love, Adventures Of A Taxi Driver, Adventures of Zatôic

In [99]:
correlation_matrix = user_ratings.corr(method='pearson')
correlation_matrix.head(10)

title,10 Items or Less,10 Things I Hate About You,15 Minutes,1984,2 Days in Paris,"20,000 Leagues Under the Sea",2001: A Space Odyssey,24 Hour Party People,25th Hour,28 Days Later,...,Young Adam,Young Frankenstein,Young and Innocent,Z,Zatoichi,Zazie dans le métro,Zodiac,eXistenZ,xXx,À nos amours
title,,,,,,,,,,,,,,,,,,,,,
10 Items or Less,1.000000,-0.012450,-0.011073,-0.011901,0.040996,0.142993,0.023696,0.083809,-0.017165,0.005946,...,0.072278,0.069160,0.099957,0.205921,0.146440,0.326914,0.166162,0.364564,0.133206,-0.009434
10 Things I Hate About You,-0.012450,1.000000,0.175260,-0.009073,-0.012890,0.047085,0.055114,0.145170,0.091828,0.298633,...,-0.012431,0.080050,0.118248,-0.009167,0.014009,-0.010265,-0.011759,-0.013000,-0.005180,0.018207
15 Minutes,-0.011073,0.175260,1.000000,0.280694,0.268258,0.183000,0.021765,0.392499,0.119882,0.194805,...,0.074966,0.070008,0.158161,0.198287,0.045516,-0.009129,0.106502,0.111338,-0.017480,0.230319
1984,-0.011901,-0.009073,0.280694,1.000000,0.165837,0.077084,0.028242,0.211639,-0.012508,0.133544,...,0.147896,0.106716,0.107475,0.287078,-0.028407,0.244305,-0.011241,-0.012427,-0.018787,0.135992
2 Days in Paris,0.040996,-0.012890,0.268258,0.165837,1.000000,0.169856,0.025719,0.180669,-0.017771,0.080364,...,0.137662,0.135356,0.145611,0.274128,0.134977,-0.013941,-0.015970,0.171909,0.024734,0.256712
"20,000 Leagues Under the Sea",0.142993,0.047085,0.183000,0.077084,0.169856,1.000000,0.058361,0.170957,0.005023,0.166427,...,0.158573,0.203896,0.110338,0.101390,0.127121,0.071874,0.076890,0.268124,0.050743,0.233218
2001: A Space Odyssey,0.023696,0.055114,0.021765,0.028242,0.025719,0.058361,1.000000,0.121294,0.097787,0.009948,...,0.280206,0.049204,0.019900,0.061908,0.058308,0.038594,0.194516,0.035789,0.000606,0.023225
24 Hour Party People,0.083809,0.145170,0.392499,0.211639,0.180669,0.170957,0.121294,1.000000,0.042293,0.088726,...,0.100101,0.162234,0.185636,0.192062,0.210311,0.176861,0.197804,0.327890,-0.013902,0.233658
25th Hour,-0.017165,0.091828,0.119882,-0.012508,-0.017771,0.005023,0.097787,0.042293,1.000000,0.050501,...,0.024221,-0.027651,0.026004,-0.012638,0.008336,0.048979,0.060128,-0.017923,-0.027096,0.213076


In [101]:
test_ratings = [("The Terminator",4),("The Dark Knight",5),("True Romance",1)]
similar_movies = pd.DataFrame()
for movie,rating in test_ratings:
    highest_ratings = correlation_matrix[movie]*(rating-2.5)
    highest_ratings = highest_ratings.sort_values(ascending=False)
    similar_movies = similar_movies.append(highest_ratings)

similar_movies.head()

,10 Items or Less,10 Things I Hate About You,15 Minutes,1984,2 Days in Paris,"20,000 Leagues Under the Sea",2001: A Space Odyssey,24 Hour Party People,25th Hour,28 Days Later,...,Young Adam,Young Frankenstein,Young and Innocent,Z,Zatoichi,Zazie dans le métro,Zodiac,eXistenZ,xXx,À nos amours
The Terminator,-0.028191,0.165583,0.134788,0.365418,0.160719,0.110676,0.165678,0.174405,-0.029629,0.037916,...,0.412123,0.200598,-0.022883,0.136916,0.086572,0.124506,0.052779,-0.029436,-0.044501,0.084313
The Dark Knight,0.149470,-0.027544,0.214595,0.316303,0.386692,0.264495,0.073026,0.160719,-0.037974,0.070571,...,0.314294,0.046029,0.170092,0.218348,0.344866,0.123233,0.382213,0.363298,0.013608,0.269187
True Romance,-0.031109,0.013853,-0.106203,-0.281077,-0.122010,-0.139292,-0.084251,-0.046619,-0.175812,-0.146685,...,-0.110496,-0.040240,-0.101479,-0.420004,-0.167889,-0.067749,-0.040561,-0.027732,0.055607,-0.271288


In [102]:
recommendations = similar_movies.sum().sort_values(ascending=False).head(12)

In [103]:
for i in recommendations.keys():
    print(i)

The Dark Knight
The Terminator
Mean Streets
War of the Worlds
Talk to Her
The Revengers' Comedies
A Beautiful Mind
Trouble in Paradise
One Flew Over the Cuckoo's Nest
Spring, Summer, Fall, Winter... and Spring
Confessions of a Dangerous Mind
Born into Brothels
